## Imports

In [155]:
import pandas as pd
import numpy as np
from ast import literal_eval

from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    cross_val_score,
    cross_validate,
    train_test_split,
)

from sklearn.preprocessing import OneHotEncoder, StandardScaler, MultiLabelBinarizer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline, make_pipeline

from sklearn.dummy import DummyRegressor
from sklearn.linear_model import Ridge, RidgeCV

from sklearn.base import TransformerMixin


In [156]:
# Solution adapted from StackOverFlow solution: https://tinyurl.com/59hmeesh
class MyMultiLabelBinarizer(TransformerMixin):
    def __init__(self, *args, **kwargs):
        self.encoder = MultiLabelBinarizer(*args, **kwargs)

    def fit(self, x, y=0):
        self.encoder.fit(x)
        return self

    def transform(self, x, y=0):
        return self.encoder.transform(x)

## Reading and Splitting Model Data

In [157]:
train_df = pd.read_csv("../data/processed/training_split.csv")
test_df = pd.read_csv("../data/processed/testing_split.csv")

categorical_list_features = ["boardgamecategory", "boardgamemechanic", "boardgamefamily", "boardgamedesigner", "boardgameartist", "boardgamepublisher"]
for feat in categorical_list_features:
    train_df[feat] = train_df[feat].apply(literal_eval)
    test_df[feat] = test_df[feat].apply(literal_eval)
    train_df[feat] = train_df[feat].apply(lambda x: set(x))
    test_df[feat] = test_df[feat].apply(lambda x: set(x))

X_train, y_train = train_df.drop(columns="average"), train_df["average"]
X_test, y_test = test_df.drop(columns="average"), test_df["average"]

## Setting Up the Column Transformers

In [169]:
numerical_features = ["yearpublished", "minplayers", "maxplayers", "playingtime", "minplaytime", "maxplaytime", "minage"]
text_feature = "description"
categorical_features = ["boardgamecategory", "boardgamemechanic", "boardgamefamily", "boardgamedesigner", "boardgameartist", "boardgamepublisher"]

preprocessor = make_column_transformer(
    (StandardScaler(), numerical_features),
    (CountVectorizer(stop_words="english"), text_feature),
    (MyMultiLabelBinarizer(), categorical_features)
)

## Testing Ridge Model

In [168]:
pipe_ridge = make_pipeline(
    preprocessor,
    Ridge()
)

param_dist = {
    "ridge__alpha": 10.0 ** np.arange(-6, 6, 1)
}

ridge_alpha_search = RandomizedSearchCV(
    pipe_ridge, param_dist, n_iter=50, n_jobs=-1, return_train_score=True
)

ridge_alpha_search.fit(X_train, y_train)

ValueError: 
All the 60 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
2 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\vikra\.conda\envs\573\lib\site-packages\joblib\parallel.py", line 822, in dispatch_one_batch
    tasks = self._ready_batches.get(block=False)
  File "c:\Users\vikra\.conda\envs\573\lib\queue.py", line 168, in get
    raise Empty
_queue.Empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\vikra\.conda\envs\573\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\vikra\.conda\envs\573\lib\site-packages\sklearn\pipeline.py", line 378, in fit
    Xt = sel

In [162]:
# cross_val_results = {}
# dummy_regressor = DummyRegressor()
# cross_val_results['dummy_regressor'] = pd.DataFrame(cross_validate(dummy_regressor, X_train, y_train, return_train_score = True)).agg(['mean', 'std']).round(3).T
# cross_val_results['dummy_regressor']

,mean,std
fit_time,0.002,0.001
score_time,0.000,0.000
test_score,-0.001,0.001
train_score,0.000,0.000
